## Problem Definition

***
The task:
- Predict loan defaults

The data:
- https://www.kaggle.com/wendykan/lending-club-loan-data/downloads/lending-club-loan-data.zip
***

## Work Plan

***
*Initial Considerations*

Generally, implementing any predictive functionality invloves:
- Understanding the business problems
- Exploring and preparing the data
- Building predictive models
- Evaluating predictive models
- Deploying the models and monitoring their performance

***

So ... <br>

What does "predicting loan defaults" mean for the business?
- let's assume it means: "limiting the losses and, at the same time, not adversely impacting the revenue generated by giving loans"

What data sources are available and what are they like?
- in this exercise there is only one data source; let's explore it

How to build the predictive models?
- first let's use scikit-learn which has the popular algorithms, such as Generalized Linear Models, Support Vector Machines, Random Forest Tress, Gradient Boosting Machines, Neural Nets, and others
- then, let's try other libs, like xgboost, lightgbm, or keras
- we will need to tune the hyperparameters, i.e. find the best ones for our models, and possibly do some feature engineering

How to evaluate the predictive models?
- the data must be split into train- and test- sets; the former will be used for model tuning, the latter only for the final validation of the model (It is crucial to hold out the test-set on the developing models until the very final validation)
- for validation we can use: Accuracy, Precision, Recall, F1, ROC, PRC, however, we'd better show some metrics that are understood by business too, such as Expected Profit (Loss) and Prediction Uncertainty (Risk)

How to deploy and monitor the predictive models?
- they can be deployed as a service which is fed with the loan characteristics data and returns the probability or some score of default; the results of predictions must be compared with the real outcomes regularly so that the model's performance can be adapted to ever changing conditions

## Imports and Configuration Settings

In [1]:
# these are some common imports and config settings

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.max_columns = 80
pd.options.display.max_rows = 20

## Split the data into train-test sets

In [2]:
# load the data

import sqlite3

with sqlite3.connect('lending-club-loan-data/database.sqlite') as db:
    df = pd.read_sql_query('SELECT * FROM loan', con=db)

df.shape

(887383, 75)

In [3]:
df.sample(1) # show one row

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
681923,215635,55091408,58662214.0,4000.0,4000.0,4000.0,36 months,12.29%,133.42,C,C1,School Bus Driver,10+ years,OWN,26500.0,Not Verified,Jul-2015,Current,n,https://www.lendingclub.com/browse/loanDetail....,None,debt_consolidation,Debt consolidation,710xx,LA,24.64,0.0,Nov-1999,1.0,NaN,90.0,6.0,1.0,2258.0,19%,14.0,w,3430.88,3430.88,781.09,781.09,569.12,211.97,0.0,0.0,0.0,Jan-2016,133.42,Feb-2016,Jan-2016,0.0,35.0,1.0,INDIVIDUAL,NaN,NaN,None,0.0,385.0,18374.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11900.0,NaN,NaN,NaN


In [4]:
# remove "index" generated by sqlite 
# and set "id" (a unique ID for the loan) as the df index, which will be useful for later investigations
df.drop('index',axis=1, inplace=True)
df = df.set_index('id')

# keep only the rows where loan_amnt is not null (you cannot default on a null loan, can you?)
df = df[ df.loan_amnt.notnull() ]

# let's assume the default also occurs when loan_status is one of the following:
# - Charged off
# - Default
# - Does not meet the credit policy. Status: Charged Off
# - Late (31-120 days)
# So,
tmp_s = df.loan_status.replace( {
    'Charged Off': 'Default',
    'Late (31-120 days)': 'Default',
    'Does not meet the credit policy. Status:Charged Off': 'Default',
    'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
})

# set "DEFAULT" as the target variable; it can take on only either 0 or 1 
df.insert(0,'DEFAULT', np.where(tmp_s == 'Default', 1, 0)) # 1 - being the DEFAULT

# drop loan_status since it's role has been taken by "DEFAULT"
df = df.drop('loan_status', axis=1)

In [5]:
# do the train-test split
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=.30)
df.shape, df_train.shape, df_test.shape

((887382, 73), (621167, 73), (266215, 73))

In [6]:
# verify that the split stratified the target variable equally
print('Train set target values incidence: \n', df_train.DEFAULT.value_counts()/df_train.shape[0] )
print('Test set target value incidence: \n', df_test.DEFAULT.value_counts()/df_test.shape[0] )

Train set target values incidence: 
 0    0.933509
1    0.066491
Name: DEFAULT, dtype: float64
Test set target value incidence: 
 0    0.9342
1    0.0658
Name: DEFAULT, dtype: float64


In [7]:
# save the train and test sets

with sqlite3.connect('lending-club-loan-data/train.sqlite') as db:
    df_train.to_sql('loan', con=db, if_exists='replace', index=True, index_label='id')

with sqlite3.connect('lending-club-loan-data/test.sqlite') as db:
    df_test.to_sql('loan', con=db, if_exists='replace', index=True, index_label='id')


###### [Next: 2 Data Exploration and Preparation](Predicting Loan Defaults - 2 Data Exploration and Preparation.ipynb)